In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dense
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE


C:\Anaconda\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Step 2: Load and Preprocess Data
data = pd.read_csv('landslidedata.csv')
attributes = ['TEMP_MAX', 'TEMP_MIN', 'PRECIPITATION', 'SPECIFIC_HUMIDITY',
              'RELATIVE_HUMIDITY', 'WIND_MIN', 'WIND_MAX',
              'EARTHQUAKE_DEPTH', 'EARTHQUAKE_MAGNITUDE' ]

In [3]:
# Normalize the data
scaler = MinMaxScaler()
data[attributes] = scaler.fit_transform(data[attributes])

In [4]:
data.tail(10)

,YEAR,MONTH,DAY,TEMP_MAX,TEMP_MIN,PRECIPITATION,RELATIVE_HUMIDITY,SPECIFIC_HUMIDITY,WIND_MIN,WIND_MAX,EARTHQUAKE_DEPTH,EARTHQUAKE_MAGNITUDE,MARKING
10947,2019,12,22,0.201843,0.213046,0.000000,0.830816,0.319088,0.008214,0.215166,0.0,0.0,0
10948,2019,12,23,0.225712,0.304552,0.000000,0.838833,0.353751,0.117043,0.224645,0.0,0.0,0
10949,2019,12,24,0.193049,0.243078,0.000000,0.833023,0.333333,0.069815,0.181043,0.0,0.0,0
10950,2019,12,25,0.155779,0.132802,0.000000,0.807576,0.290123,0.162218,0.234123,0.0,0.0,0
10951,2019,12,26,0.234925,0.233224,0.015398,0.755287,0.321937,0.045175,0.232227,0.0,0.0,0
10952,2019,12,27,0.224456,0.298921,0.034565,0.929584,0.405983,0.227926,0.261611,0.0,0.0,1
10953,2019,12,28,0.158710,0.259972,0.000796,0.883105,0.339031,0.232033,0.233175,0.0,0.0,0
10954,2019,12,29,0.248325,0.162834,0.000159,0.827911,0.336182,0.209446,0.284360,0.0,0.0,0
10955,2019,12,30,0.239531,0.235101,0.000000,0.832210,0.371320,0.039014,0.298578,0.0,0.0,0
10956,2019,12,31,0.306114,0.392304,0.000000,0.840228,0.426401,0.151951,0.185782,0.0,0.0,0


In [5]:
# Split data into training and testing sets
X = data[attributes].values
y = data['MARKING'].values  # Assuming you have a 'LANDSLIDE_LABEL' column for labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [6]:
unique, count = np.unique(y_train, return_counts=True)
y_train_dict_value_count = {k:v for (k,v) in zip(unique,count)}
y_train_dict_value_count

{0: 5297, 1: 2372}

In [7]:
sm = SMOTE(random_state =12, sampling_strategy = 1.0)
# x_train_result, y_train_result = sm.fit_sample(X_train, y_train)
x_train_result, y_train_result = sm.fit_resample(X_train, y_train)

In [8]:
unique, count = np.unique(y_train_result, return_counts=True)
y_train_smote_value_count = {k:v for (k,v) in zip(unique,count)}
y_train_smote_value_count

{0: 5297, 1: 5297}

In [9]:
# Reshape the data for LSTM input
x_train_result = x_train_result.reshape(x_train_result.shape[0], x_train_result.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense

In [11]:
# Define the model
bilstm = Sequential()

In [12]:
# Add the Bidirectional LSTM layer with 128 units and 'relu' activation
bilstm.add(Bidirectional(LSTM(128, activation='relu', input_shape=(x_train_result.shape[1], 1))))

# Add more layers
bilstm.add(Dense(64, activation='relu'))  # Adding a Dense layer with 64 units and 'relu' activation
bilstm.add(Dense(32, activation='relu'))  # Adding another Dense layer with 32 units and 'relu' activation

# Output layer
bilstm.add(Dense(1, activation='sigmoid'))  # Output layer with 1 unit and 'sigmoid' activation

# Compile the model
bilstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
# Train the model
bilstm.fit(x_train_result, y_train_result, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
332/332 [==============================] - 18s 28ms/step - loss: 0.3893 - accuracy: 0.8145 - val_loss: 0.2209 - val_accuracy: 0.8908
Epoch 2/50
332/332 [==============================] - 9s 28ms/step - loss: 0.1776 - accuracy: 0.9285 - val_loss: 0.4524 - val_accuracy: 0.8476
Epoch 3/50
332/332 [==============================] - 9s 27ms/step - loss: 0.1608 - accuracy: 0.9425 - val_loss: 0.0894 - val_accuracy: 0.9662
Epoch 4/50
332/332 [==============================] - 10s 31ms/step - loss: 0.1352 - accuracy: 0.9478 - val_loss: 0.1379 - val_accuracy: 0.9304
Epoch 5/50
332/332 [==============================] - 8s 25ms/step - loss: 0.1245 - accuracy: 0.9543 - val_loss: 0.1652 - val_accuracy: 0.9164
Epoch 6/50
332/332 [==============================] - 8s 25ms/step - loss: 0.1017 - accuracy: 0.9591 - val_loss: 0.0757 - val_accuracy: 0.9778
Epoch 7/50
332/332 [==============================] - 10s 29ms/step - loss: 0.0862 - accuracy: 0.9659 - val_loss: 0.0933 - val_accuracy: 0.9

In [14]:
# Evaluate the model
loss, accuracy = bilstm.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy*100:.2f}%')

103/103 [==============================] - 1s 12ms/step - loss: 0.0485 - accuracy: 0.9769
Accuracy: 97.69%


In [15]:
import pickle 

In [16]:
with open('bilstm.pickle', 'wb') as f:
    pickle.dump(bilstm , f)


In [17]:
with open('bilstm.pickle', 'rb') as f:
    loaded_model = pickle.load(f)

In [18]:
print(loaded_model)

In [19]:
# Initialize a flag to continue taking input
continue_input = True

while continue_input:
    new_attributes = []

    print("Enter new attributes:")
    for attribute in attributes:
        value = float(input(f"Enter value for {attribute}: "))
        new_attributes.append(value)

    # Normalize the new attributes using the same scaler
    new_attributes = np.array(new_attributes).reshape(1, -1)
    new_attributes = scaler.transform(new_attributes)

    # Reshape the new attributes for LSTM input
    new_attributes = new_attributes.reshape(1, new_attributes.shape[1], 1)

    # Predict marking for new attributes
    predicted_prob = loaded_model.predict(new_attributes)
    predicted_marking = "Landslide" if predicted_prob >= 0.5 else "No Landslide"
    print("Predicted Marking:", predicted_marking)

    # Ask if user wants to continue
    user_input = input("Do you want to continue? (yes/no): ")
    if user_input.lower() != 'yes':
        continue_input = False


Enter new attributes:
Enter value for TEMP_MAX: 33
Enter value for TEMP_MIN: 25
Enter value for PRECIPITATION: 2.3
Enter value for SPECIFIC_HUMIDITY: 75
Enter value for RELATIVE_HUMIDITY: 12
Enter value for WIND_MIN: 2.3
Enter value for WIND_MAX: 3.1
Enter value for EARTHQUAKE_DEPTH: 0
Enter value for EARTHQUAKE_MAGNITUDE: 0


C:\Anaconda\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 1s 847ms/step
Predicted Marking: Landslide
Do you want to continue? (yes/no): NO
